In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
import statistics
import pymysql
from sqlalchemy import create_engine
import getpass  
warnings.filterwarnings('ignore')
from collections import Counter
from seaborn import load_dataset
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LinearRegression as LinReg
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error as mae
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix 
from sklearn import neighbors 
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE

In [2]:
Hotels_cleaned = pd.read_csv('Hotels_cleaned.csv')

In [3]:
Hotels_cleaned.shape

(441, 19)

In [4]:
Other_lodge_cleaned = pd.read_csv('Other_Lodge_cleaned.csv')

In [5]:
Other_lodge_cleaned.shape

(210, 19)

In [6]:
Open_Hotels_cleaned = pd.read_csv('Cleaned_Open_Hotels.csv')

In [7]:
Open_Hotels_cleaned.shape

(249, 4)

In [8]:
Hotels_Lodge=pd.concat([Hotels_cleaned, Other_lodge_cleaned]).reset_index()

In [9]:
Hotels_Lodge.shape

(651, 20)

In [10]:
Hotels_Lodge.head(5)

,index,name,created,addresses_road_name,addresses_start_street_number,addresses_neighborhood_id,addresses_neighborhood_name,addresses_district_id,addresses_district_name,addresses_zip_code,addresses_town,type_of_contact,phone_number,hotel_Rating,geo_epgs_25831_x,geo_epgs_25831_y,geo_epgs_4326_x,geo_epgs_4326_y,month_created,year_created
0,0,Hotel Balmoral,1984-05-29 22:00:00+00:00,Via Augusta,5,26,Sant Gervasi - Galvany,5,Sarrià-Sant Gervasi,8006,BARCELONA,Centraleta,932178700,4,42937703921294704,4583085639865460,4139620718347950,2155190161506960,5,1984
1,1,Hotel Fornos,1984-05-29 22:00:00+00:00,C Rambla,44,2,el Barri Gòtic,1,Ciutat Vella,8002,BARCELONA,Tel.,933182024,2,43100882773755600,4581225140356750,4137959188566090,21749199534758200,5,1984
2,2,Hotel Granvia,1984-05-29 22:00:00+00:00,G.V. Corts Catalanes,642,7,la Dreta de l'Eixample,2,Eixample,8007,BARCELONA,Tel.,933181900,3,43056889077459200,4582390495894150,4139005003654940,2169525637778740,5,1984
3,3,Hotel Nouvel,1984-05-29 22:00:00+00:00,C Santa Anna,18,2,el Barri Gòtic,1,Ciutat Vella,8002,BARCELONA,Tel.,933018274,3,43073264629176704,4581861958085160,4138530378088550,21715445758064500,5,1984
4,4,Hotel Silken Sant Gervasi,2000-05-17 22:00:00+00:00,C Sant Gervasi de Cassoles,26,25,Sant Gervasi - la Bonanova,5,Sarrià-Sant Gervasi,8022,BARCELONA,Centraleta,932531740,3,42802010219409800,4584043072241010,41404710071918704,21388460431523300,5,2000


In [11]:
Hotels_Lodge.tail(5)

,index,name,created,addresses_road_name,addresses_start_street_number,addresses_neighborhood_id,addresses_neighborhood_name,addresses_district_id,addresses_district_name,addresses_zip_code,addresses_town,type_of_contact,phone_number,hotel_Rating,geo_epgs_25831_x,geo_epgs_25831_y,geo_epgs_4326_x,geo_epgs_4326_y,month_created,year_created
646,205,360 Hostel Barcelona,2015-07-21 10:08:00+00:00,C Bailèn,7,7,la Dreta de l'Eixample,2,Eixample,8010,BARCELONA,Tel.,932 469 973,0,4312355242606340,4582559606380390,4139163043193780,21774793346997400,7,2015
647,206,Apartament Turistic,2021-02-24 14:46:11.767956+00:00,Antic de Bofarull,11,64,el Camp de l'Arpa del Clot,10,Sant Martí,8026,Barcelona,NaN,NaN,0,43198261956047,458467785675468,4141077240375480,2186176787572690,2,2021
648,207,Residencia d'Estudiants Vita Student,2022-01-20 09:00:25.443196+00:00,Avinguda d'Esplugues,102,21,Pedralbes,4,Les Corts,8034,Barcelona,Tel.,93 522 01 79,0,42576576072968,4582776248770940,4139309538488680,21120339123496700,1,2022
649,208,Residencia d'Estudiants LIV Student Sarria,2022-07-22 10:21:53.890768+00:00,Carrer de Can Ràbia,7,24,les Tres Torres,5,Sarrià-Sant Gervasi,8017,Barcelona,Tel.,689954120,0,427631216,4582742042,4139295732192430,21343495003610500,7,2022
650,209,Residencia d'Estudiants LIV Student Barcelona 22@,2022-09-01 07:24:12.416589+00:00,Carrer de Puigcerdà,90,70,el Besòs i el Maresme,10,Sant Martí,8019,Barcelona,Tel.,938 679 000,0,434185312,4584925457,4141318586098700,22125024867898000,9,2022


In [12]:
Hotels_Lodge_Open=pd.merge(left=Hotels_Lodge, right=Open_Hotels_cleaned,how='outer', left_on='name', right_on='name').reset_index()

In [13]:
Hotels_Lodge_Open.head(60)

,level_0,index,name,created,addresses_road_name,addresses_start_street_number,addresses_neighborhood_id,addresses_neighborhood_name,addresses_district_id,addresses_district_name,...,hotel_Rating,geo_epgs_25831_x,geo_epgs_25831_y,geo_epgs_4326_x,geo_epgs_4326_y,month_created,year_created,Categoria,Municipi,Open
0,0,0.0,Hotel Balmoral,1984-05-29 22:00:00+00:00,Via Augusta,5.0,26.0,Sant Gervasi - Galvany,5.0,Sarrià-Sant Gervasi,...,4.0,4.293770e+16,4.583086e+15,4.139621e+15,2.155190e+15,5.0,1984.0,4.0,Barcelona,Si
1,1,1.0,Hotel Fornos,1984-05-29 22:00:00+00:00,C Rambla,44.0,2.0,el Barri Gòtic,1.0,Ciutat Vella,...,2.0,4.310088e+16,4.581225e+15,4.137959e+15,2.174920e+16,5.0,1984.0,NaN,NaN,NaN
2,2,2.0,Hotel Granvia,1984-05-29 22:00:00+00:00,G.V. Corts Catalanes,642.0,7.0,la Dreta de l'Eixample,2.0,Eixample,...,3.0,4.305689e+16,4.582390e+15,4.139005e+15,2.169526e+15,5.0,1984.0,NaN,NaN,NaN
3,3,3.0,Hotel Nouvel,1984-05-29 22:00:00+00:00,C Santa Anna,18.0,2.0,el Barri Gòtic,1.0,Ciutat Vella,...,3.0,4.307326e+16,4.581862e+15,4.138530e+15,2.171545e+16,5.0,1984.0,NaN,NaN,NaN
4,4,4.0,Hotel Silken Sant Gervasi,2000-05-17 22:00:00+00:00,C Sant Gervasi de Cassoles,26.0,25.0,Sant Gervasi - la Bonanova,5.0,Sarrià-Sant Gervasi,...,3.0,4.280201e+16,4.584043e+15,4.140471e+16,2.138846e+16,5.0,2000.0,3.0,Barcelona,Si
5,5,5.0,BCN Urban Hotels Gran Ronda,2005-11-16 10:34:34+00:00,Rda Sant Antoni,49.0,10.0,Sant Antoni,2.0,Eixample,...,3.0,4.300594e+15,4.581631e+15,4.138316e+15,2.163519e+16,11.0,2005.0,NaN,NaN,NaN
6,6,6.0,Hotel Villa Emilia by Katja & Antoine,2006-11-13 15:40:51+00:00,C Calàbria,115.0,9.0,la Nova Esquerra de l'Eixample,2.0,Eixample,...,4.0,4.292699e+15,4.581319e+15,4.138028e+15,2.154115e+16,11.0,2006.0,4.0,Barcelona,Si
7,7,7.0,Hotel HG City Suites Barcelona,1997-09-30 22:00:00+00:00,Via Augusta,89.0,26.0,Sant Gervasi - Galvany,5.0,Sarrià-Sant Gervasi,...,4.0,4.289508e+15,4.583607e+15,4.140087e+16,2.150030e+15,9.0,1997.0,NaN,NaN,NaN
8,8,8.0,Hotel HCC Taber,1996-09-16 22:00:00+00:00,C Aragó,256.0,7.0,la Dreta de l'Eixample,2.0,Eixample,...,3.0,4.300349e+16,4.582464e+16,4.139066e+15,2.163131e+16,9.0,1996.0,NaN,NaN,NaN
9,9,9.0,Hotel Evenia Rocafort,2003-12-03 12:24:03+00:00,C Rocafort,25.0,10.0,Sant Antoni,2.0,Eixample,...,3.0,4.295223e+15,4.580870e+15,4.137626e+15,2.157185e+16,12.0,2003.0,NaN,NaN,NaN


In [14]:
Hotels_Lodge_Open.shape

(708, 24)

In [15]:
Hotels_Lodge_Open['Open']=Hotels_Lodge_Open['Open'].fillna('No')

In [16]:
Hotels_Lodge_Open.head(5)

,level_0,index,name,created,addresses_road_name,addresses_start_street_number,addresses_neighborhood_id,addresses_neighborhood_name,addresses_district_id,addresses_district_name,...,hotel_Rating,geo_epgs_25831_x,geo_epgs_25831_y,geo_epgs_4326_x,geo_epgs_4326_y,month_created,year_created,Categoria,Municipi,Open
0,0,0.0,Hotel Balmoral,1984-05-29 22:00:00+00:00,Via Augusta,5.0,26.0,Sant Gervasi - Galvany,5.0,Sarrià-Sant Gervasi,...,4.0,4.293770e+16,4.583086e+15,4.139621e+15,2.155190e+15,5.0,1984.0,4.0,Barcelona,Si
1,1,1.0,Hotel Fornos,1984-05-29 22:00:00+00:00,C Rambla,44.0,2.0,el Barri Gòtic,1.0,Ciutat Vella,...,2.0,4.310088e+16,4.581225e+15,4.137959e+15,2.174920e+16,5.0,1984.0,NaN,NaN,No
2,2,2.0,Hotel Granvia,1984-05-29 22:00:00+00:00,G.V. Corts Catalanes,642.0,7.0,la Dreta de l'Eixample,2.0,Eixample,...,3.0,4.305689e+16,4.582390e+15,4.139005e+15,2.169526e+15,5.0,1984.0,NaN,NaN,No
3,3,3.0,Hotel Nouvel,1984-05-29 22:00:00+00:00,C Santa Anna,18.0,2.0,el Barri Gòtic,1.0,Ciutat Vella,...,3.0,4.307326e+16,4.581862e+15,4.138530e+15,2.171545e+16,5.0,1984.0,NaN,NaN,No
4,4,4.0,Hotel Silken Sant Gervasi,2000-05-17 22:00:00+00:00,C Sant Gervasi de Cassoles,26.0,25.0,Sant Gervasi - la Bonanova,5.0,Sarrià-Sant Gervasi,...,3.0,4.280201e+16,4.584043e+15,4.140471e+16,2.138846e+16,5.0,2000.0,3.0,Barcelona,Si


In [17]:
Hotels_Lodge_Open.drop(['Categoria','Municipi','index'], inplace=True, axis=1)

In [18]:
Hotels_Lodge_Open.head(5)

,level_0,name,created,addresses_road_name,addresses_start_street_number,addresses_neighborhood_id,addresses_neighborhood_name,addresses_district_id,addresses_district_name,addresses_zip_code,...,type_of_contact,phone_number,hotel_Rating,geo_epgs_25831_x,geo_epgs_25831_y,geo_epgs_4326_x,geo_epgs_4326_y,month_created,year_created,Open
0,0,Hotel Balmoral,1984-05-29 22:00:00+00:00,Via Augusta,5.0,26.0,Sant Gervasi - Galvany,5.0,Sarrià-Sant Gervasi,8006.0,...,Centraleta,932178700,4.0,4.293770e+16,4.583086e+15,4.139621e+15,2.155190e+15,5.0,1984.0,Si
1,1,Hotel Fornos,1984-05-29 22:00:00+00:00,C Rambla,44.0,2.0,el Barri Gòtic,1.0,Ciutat Vella,8002.0,...,Tel.,933182024,2.0,4.310088e+16,4.581225e+15,4.137959e+15,2.174920e+16,5.0,1984.0,No
2,2,Hotel Granvia,1984-05-29 22:00:00+00:00,G.V. Corts Catalanes,642.0,7.0,la Dreta de l'Eixample,2.0,Eixample,8007.0,...,Tel.,933181900,3.0,4.305689e+16,4.582390e+15,4.139005e+15,2.169526e+15,5.0,1984.0,No
3,3,Hotel Nouvel,1984-05-29 22:00:00+00:00,C Santa Anna,18.0,2.0,el Barri Gòtic,1.0,Ciutat Vella,8002.0,...,Tel.,933018274,3.0,4.307326e+16,4.581862e+15,4.138530e+15,2.171545e+16,5.0,1984.0,No
4,4,Hotel Silken Sant Gervasi,2000-05-17 22:00:00+00:00,C Sant Gervasi de Cassoles,26.0,25.0,Sant Gervasi - la Bonanova,5.0,Sarrià-Sant Gervasi,8022.0,...,Centraleta,932531740,3.0,4.280201e+16,4.584043e+15,4.140471e+16,2.138846e+16,5.0,2000.0,Si


In [19]:
Hotels_Lodge_Open.drop(['level_0'], inplace=True, axis=1)

In [20]:
Hotels_Lodge_Open.head(5)

,name,created,addresses_road_name,addresses_start_street_number,addresses_neighborhood_id,addresses_neighborhood_name,addresses_district_id,addresses_district_name,addresses_zip_code,addresses_town,type_of_contact,phone_number,hotel_Rating,geo_epgs_25831_x,geo_epgs_25831_y,geo_epgs_4326_x,geo_epgs_4326_y,month_created,year_created,Open
0,Hotel Balmoral,1984-05-29 22:00:00+00:00,Via Augusta,5.0,26.0,Sant Gervasi - Galvany,5.0,Sarrià-Sant Gervasi,8006.0,BARCELONA,Centraleta,932178700,4.0,4.293770e+16,4.583086e+15,4.139621e+15,2.155190e+15,5.0,1984.0,Si
1,Hotel Fornos,1984-05-29 22:00:00+00:00,C Rambla,44.0,2.0,el Barri Gòtic,1.0,Ciutat Vella,8002.0,BARCELONA,Tel.,933182024,2.0,4.310088e+16,4.581225e+15,4.137959e+15,2.174920e+16,5.0,1984.0,No
2,Hotel Granvia,1984-05-29 22:00:00+00:00,G.V. Corts Catalanes,642.0,7.0,la Dreta de l'Eixample,2.0,Eixample,8007.0,BARCELONA,Tel.,933181900,3.0,4.305689e+16,4.582390e+15,4.139005e+15,2.169526e+15,5.0,1984.0,No
3,Hotel Nouvel,1984-05-29 22:00:00+00:00,C Santa Anna,18.0,2.0,el Barri Gòtic,1.0,Ciutat Vella,8002.0,BARCELONA,Tel.,933018274,3.0,4.307326e+16,4.581862e+15,4.138530e+15,2.171545e+16,5.0,1984.0,No
4,Hotel Silken Sant Gervasi,2000-05-17 22:00:00+00:00,C Sant Gervasi de Cassoles,26.0,25.0,Sant Gervasi - la Bonanova,5.0,Sarrià-Sant Gervasi,8022.0,BARCELONA,Centraleta,932531740,3.0,4.280201e+16,4.584043e+15,4.140471e+16,2.138846e+16,5.0,2000.0,Si


In [21]:
Hotels_Lodge_Open.shape

(708, 20)

In [22]:
Hotels_Lodge_Open.isna().sum()

name                              0
created                          57
addresses_road_name              57
addresses_start_street_number    57
addresses_neighborhood_id        57
addresses_neighborhood_name      57
addresses_district_id            57
addresses_district_name          57
addresses_zip_code               57
addresses_town                   57
type_of_contact                  59
phone_number                     59
hotel_Rating                     57
geo_epgs_25831_x                 57
geo_epgs_25831_y                 57
geo_epgs_4326_x                  57
geo_epgs_4326_y                  57
month_created                    57
year_created                     57
Open                              0
dtype: int64

In [23]:
#Hotels_Lodge_Open=Hotels_Lodge_Open.dropna(how='all')
Hotels_Lodge_Open=Hotels_Lodge_Open.dropna()

In [24]:
Hotels_Lodge_Open.isna().sum()

name                             0
created                          0
addresses_road_name              0
addresses_start_street_number    0
addresses_neighborhood_id        0
addresses_neighborhood_name      0
addresses_district_id            0
addresses_district_name          0
addresses_zip_code               0
addresses_town                   0
type_of_contact                  0
phone_number                     0
hotel_Rating                     0
geo_epgs_25831_x                 0
geo_epgs_25831_y                 0
geo_epgs_4326_x                  0
geo_epgs_4326_y                  0
month_created                    0
year_created                     0
Open                             0
dtype: int64

In [25]:
Hotels_Lodge_Open.shape

(649, 20)

In [26]:
Hotels_Lodge_Open.dtypes

name                              object
created                           object
addresses_road_name               object
addresses_start_street_number    float64
addresses_neighborhood_id        float64
addresses_neighborhood_name       object
addresses_district_id            float64
addresses_district_name           object
addresses_zip_code               float64
addresses_town                    object
type_of_contact                   object
phone_number                      object
hotel_Rating                     float64
geo_epgs_25831_x                 float64
geo_epgs_25831_y                 float64
geo_epgs_4326_x                  float64
geo_epgs_4326_y                  float64
month_created                    float64
year_created                     float64
Open                              object
dtype: object

In [27]:
Hotels_Lodge_Open['created'] = pd.to_datetime(Hotels_Lodge_Open['created'], format = "%Y-%m-%dT%H:%M:%S%z", errors='ignore',utc=True)

In [28]:
Hotels_Lodge_Open.shape

(649, 20)

In [29]:
Hotels_Lodge_Open['addresses_start_street_number'] = Hotels_Lodge_Open['addresses_start_street_number'].astype(int)

In [30]:
Hotels_Lodge_Open['addresses_district_id'] = Hotels_Lodge_Open['addresses_district_id'].astype(int)

In [31]:
Hotels_Lodge_Open['addresses_zip_code'] = Hotels_Lodge_Open['addresses_zip_code'].astype(int)

In [32]:
Hotels_Lodge_Open['hotel_Rating'] = Hotels_Lodge_Open['hotel_Rating'].astype(int)

In [33]:
Hotels_Lodge_Open['geo_epgs_25831_x'] = Hotels_Lodge_Open['geo_epgs_25831_x'].astype(int)

In [34]:
Hotels_Lodge_Open['geo_epgs_25831_y'] = Hotels_Lodge_Open['geo_epgs_25831_y'].astype(int)

In [35]:
Hotels_Lodge_Open['geo_epgs_4326_x'] = Hotels_Lodge_Open['geo_epgs_4326_x'].astype(int)

In [36]:
Hotels_Lodge_Open['geo_epgs_4326_y'] = Hotels_Lodge_Open['geo_epgs_4326_y'].astype(int)

In [37]:
Hotels_Lodge_Open['month_created'] = Hotels_Lodge_Open['month_created'].astype(int)

In [38]:
Hotels_Lodge_Open['year_created'] = Hotels_Lodge_Open['year_created'].astype(int)

In [39]:
Hotels_Lodge_Open['addresses_neighborhood_id'] = Hotels_Lodge_Open['addresses_neighborhood_id'].astype(int)

In [40]:
Hotels_Lodge_Open.head(5)

,name,created,addresses_road_name,addresses_start_street_number,addresses_neighborhood_id,addresses_neighborhood_name,addresses_district_id,addresses_district_name,addresses_zip_code,addresses_town,type_of_contact,phone_number,hotel_Rating,geo_epgs_25831_x,geo_epgs_25831_y,geo_epgs_4326_x,geo_epgs_4326_y,month_created,year_created,Open
0,Hotel Balmoral,1984-05-29 22:00:00+00:00,Via Augusta,5,26,Sant Gervasi - Galvany,5,Sarrià-Sant Gervasi,8006,BARCELONA,Centraleta,932178700,4,-2147483648,-2147483648,-2147483648,-2147483648,5,1984,Si
1,Hotel Fornos,1984-05-29 22:00:00+00:00,C Rambla,44,2,el Barri Gòtic,1,Ciutat Vella,8002,BARCELONA,Tel.,933182024,2,-2147483648,-2147483648,-2147483648,-2147483648,5,1984,No
2,Hotel Granvia,1984-05-29 22:00:00+00:00,G.V. Corts Catalanes,642,7,la Dreta de l'Eixample,2,Eixample,8007,BARCELONA,Tel.,933181900,3,-2147483648,-2147483648,-2147483648,-2147483648,5,1984,No
3,Hotel Nouvel,1984-05-29 22:00:00+00:00,C Santa Anna,18,2,el Barri Gòtic,1,Ciutat Vella,8002,BARCELONA,Tel.,933018274,3,-2147483648,-2147483648,-2147483648,-2147483648,5,1984,No
4,Hotel Silken Sant Gervasi,2000-05-17 22:00:00+00:00,C Sant Gervasi de Cassoles,26,25,Sant Gervasi - la Bonanova,5,Sarrià-Sant Gervasi,8022,BARCELONA,Centraleta,932531740,3,-2147483648,-2147483648,-2147483648,-2147483648,5,2000,Si


In [41]:
Hotels_Lodge_Open.Open.value_counts(dropna=False)

No    459
Si    190
Name: Open, dtype: int64

In [42]:
Hotels_Lodge_Open.to_csv('Merged_Hotels_Lodge_Open.csv', index=False,sep=',',encoding='utf-8')